In [1]:
from hypergraphs import *
from kegg import *

In [2]:
superpathwayDict = {
    "Carbohydrate metabolism":
[
	("00010", "Glycolysis / Gluconeogenesis"),
	("00020", "Citrate cycle (TCA cycle)"),
	("00030", "Pentose phosphate pathway"),
	("00040", "Pentose and glucuronate interconversions"),
	("00051", "Fructose and mannose metabolism"),
	("00052", "Galactose metabolism"),
	("00053", "Ascorbate and aldarate metabolism"),
	("00500", "Starch and sucrose metabolism"),
	("00520", "Amino sugar and nucleotide sugar metabolism"),
	("00620", "Pyruvate metabolism"),
	("00630", "Glyoxylate and dicarboxylate metaboliscm"),
	("00640", "Propanoate metabolism"),
	("00650", "Butanoate metabolism"),
	("00562", "Inositol phosphate metabolism")
],
"Energy metabolism":
[
	("00190", "Oxidative phosphorylation"),
	("00910", "Nitrogen metabolism"),
	("00920", "Sulfur metabolism")
],
"Lipid metabolism":
[	("00061", "Fatty acid biosynthesis"),
	("00062", "Fatty acid elongation"),	
	("00071", "Fatty acid degradation"),
	("00100", "Steroid biosynthesis"),	
	("00120", "Primary bile acid biosynthesis"),
	("00140", "Steroid hormone biosynthesis"),
	("00561", "Glycerolipid metabolism"),
	("00564", "Glycerophospholipid metabolism"),
	("00565", "Ether lipid metabolism"),
	("00600", "Sphingolipid metabolism"),
	("00590", "Arachidonic acid metabolism"),
	("00591", "Linoleic acid metabolism"),
	("00592", "alpha-Linolenic acid metabolism"),
	("01040", "Biosynthesis of unsaturated fatty acids")
],
"Nucleotide metabolism":
[
	("00230", "Purine metabolism"),
	("00240", "Pyrimidine metabolism")
],
"Amino acid metabolism":
[
	("00250", "Alanine, aspartate and glutamate metabolism"),
	("00260", "Glycine, serine and threonine metabolism"),
	("00270", "Cysteine and methionine metabolism"),
	("00280", "Valine, leucine and isoleucine degradation"),
	("00290", "Valine, leucine and isoleucine biosynthesis"),
	("00310", "Lysine degradation"),
	("00220", "Arginine biosynthesis"),
	("00330", "Arginine and proline metabolism"),
	("00340", "Histidine metabolism"),
	("00350", "Tyrosine metabolism"),
	("00360", "Phenylalanine metabolism"),
	("00380", "Tryptophan metabolism"),
	("00400", "Phenylalanine, tyrosine and tryptophan biosynthesis")
],
"Metabolism of other amino acids":
[
	("00410", "beta-Alanine metabolism"),
	("00430", "Taurine and hypotaurine metabolism"),
	("00440", "Phosphonate and phosphinate metabolism"),
	("00450", "Selenocompound metabolism"),
	("00470", "D-Amino acid metabolism"),
	("00480", "Glutathione metabolism")
],
"Glycan biosynthesis and metabolism":
[
	("00510", "N-Glycan biosynthesis"),
	("00513", "Various types of N-glycan biosynthesis"),
	("00512", "Mucin type O-glycan biosynthesis"),
	("00515", "Mannose type O-glycan biosynthesis"),
	("00514", "Other types of O-glycan biosynthesis"),
	("00532", "Glycosaminoglycan biosynthesis - chondroitin sulfate / dermatan sulfate"),
	("00534", "Glycosaminoglycan biosynthesis - heparan sulfate / heparin"),
	("00533", "Glycosaminoglycan biosynthesis - keratan sulfate"),
	("00531", "Glycosaminoglycan degradation"),
	("00563", "Glycosylphosphatidylinositol (GPI)-anchor biosynthesis"),
	("00601", "Glycosphingolipid biosynthesis - lacto and neolacto series"),
	("00603", "Glycosphingolipid biosynthesis - globo and isoglobo series"),
	("00604", "Glycosphingolipid biosynthesis - ganglio series"),
	("00511", "Other glycan degradation")
],
"Metabolism of cofactors and vitamins":
[
	("00730", "Thiamine metabolism"),
	("00740", "Riboflavin metabolism"),
	("00750", "Vitamin B6 metabolism"),
	("00760", "Nicotinate and nicotinamide metabolism"),
	("00770", "Pantothenate and CoA biosynthesis"),
	("00780", "Biotin metabolism"),
	("00785", "Lipoic acid metabolism"),
	("00790", "Folate biosynthesis"),
	("00670", "One carbon pool by folate"),
	("00830", "Retinol metabolism"),
	("00860", "Porphyrin metabolism"),
	("00130", "Ubiquinone and other terpenoid-quinone biosynthesis")
],
"Metabolism of terpenoids and polyketides":
[
	("00900", "Terpenoid backbone biosynthesis")
],
"Biosynthesis of other secondary metabolites":
[
	("00232", "Caffeine metabolism"),
	("00524", "Neomycin, kanamycin and gentamicin biosynthesis")
],
"Xenobiotics biodegradation and metabolism":
[
	("00980", "Metabolism of xenobiotics by cytochrome P450"),
	("00982", "Drug metabolism - cytochrome P450"),
	("00983", "Drug metabolism - other enzymes")
]
}

In [3]:
organismDict = {"mmu": "Mus musculus (house mouse)", "dre": "Danio rerio (zebrafish)", "nfu": "Nothobranchius furzeri (turquoise killifish)"}

In [4]:
outputDirectory = "/Users/boldi/Desktop/pw/"
dataDirectory = "../../LaTeX/Data/KEGG-Pathways/"

In [5]:
for organism in organismDict.keys():
    for superpathway in superpathwayDict.keys():
        safeSuperpathway = makesafe(superpathway)
        H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML(
            [os.path.join(dataDirectory, organism, f"{organism}{x[0]}-sbml.xml") for x in superpathwayDict[superpathway]])
        dd=hyper_cardon_crochemore(H, proxy=sr2r)
        
        groupedReactions = mapKeys(dd, prefOnly="rn")
        r2gr = {}
        r2grall = {}
        for i, reactions in enumerate(groupedReactions):
            for reaction in reactions:
                r2gr[reaction] = i
                r2grall[reaction] = reactions
                
        groupedCompounds = mapKeys(dd, prefOnly="C")
        c2gc = {}
        c2gcall = {}
        for i, compounds in enumerate(groupedCompounds):
            for compound in compounds:
                c2gc[compound] = i + len(groupedReactions)
                c2gcall[compound] = compounds
                
        writeKEGGHtml(dd, os.path.join(outputDirectory, organism, safeSuperpathway), safeSuperpathway + "R", 
              f"{organismDict[organism]}, {superpathway}, Reactions group",
              onlyNonTrivial=True, prefOnly="r", cutPref="rn")
        index = [(f"{safeSuperpathway}R-cont-0.html", "Reaction groups")]
        writeKEGGHtml(dd, os.path.join(outputDirectory, organism, safeSuperpathway), safeSuperpathway + "C", 
              f"{organismDict[organism]}, {superpathway}, Compound group",
              onlyNonTrivial=True, prefOnly="C", cutPref="")
        index += [(f"{safeSuperpathway}C-cont-0.html", "Compound groups")]
        
        index += writeKEGGpdf(outputDirectory, organism, superpathway, superpathwayDict, r2gr, r2grall, c2gc, c2gcall)
        
        writeIndex(os.path.join(outputDirectory, organism, safeSuperpathway), index, 
           f"{organismDict[organism]}, {superpathway}")
        
        with open(os.path.join(outputDirectory, organism, safeSuperpathway, "data.pkl"), "wb") as file:
            pickle.dump([dd, sr2r, r2ss, rid2rname, mid2mname], file)
    
    writeIndex(os.path.join(outputDirectory, organism), 
           [("{}/index.html".format(makesafe(k)), k) for k in superpathwayDict.keys()],
           f"{organismDict[organism]}")
    
writeIndex(os.path.join(outputDirectory), 
          [(f"{k}/index.html", v) for k,v in organismDict.items()],
           "Pathways")

/Users/boldi/.local/share/virtualenvs/qf-BIT_cIX4/lib/python3.8/site-packages/hypernetx/classes/hypergraph.py:968: UserWarning: Cannot add edge. Edge already in hypergraph
  warnings.warn("Cannot add edge. Edge already in hypergraph")


Pathway ('00190', 'Oxidative phosphorylation') could not be downloaded: ignoring



KeyboardInterrupt



In [ ]:
H.nodes

In [6]:
organism="mmu"
superpathway="Lipid metabolism"
H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML(
            [os.path.join(dataDirectory, organism, f"{organism}{x[0]}-sbml.xml") for x in superpathwayDict[superpathway]])

In [7]:
H.nodes

EntitySet(:Nodes,['C02373', 'C04233', 'C00695', 'C05465', 'C21072', 'C06423', 'C02679', 'C05290', 'C02538', 'C00735', 'C00040_1', 'C00422', 'C06126', 'C00674', 'C21939', 'C11508', 'C02041', 'C05444', 'C01561', 'C00951', 'C04317', 'C05439', 'C05485', 'C21935', 'C03681', 'C00535', 'C04554', 'C14774', 'C05744_1', 'C05446', 'C06424', 'C00570', 'C00410', 'C05479', 'C03917', 'C14748', 'C01194', 'C16162_1', 'C15647', 'C05107', 'C05749', 'C13712', 'C12145', 'C00184', 'C14772', 'C04742', 'C01953', 'C17335', 'C00751', 'C06425', 'C01144', 'C05977', 'C15783', 'C15614', 'C14771', 'C01189', 'C00065', 'C14812', 'C05466', 'C14775', 'C15781', 'C01885', 'C05262', 'C05266', 'C05302', 'C00093', 'C03852', 'C22136', 'C00249', 'C06124', 'C06127', 'C01802', 'C05747', 'C05285', 'C04633', 'C01724', 'C16167', 'C16335', 'C05744', 'C00712', 'C01944', 'C00136', 'C05755', 'C05452', 'C05447', 'C03819', 'C00468', 'C04308', 'C00226', 'C00836', 'C05301', 'C05259', 'C03820', 'C17345', 'C05973', 'C00040', 'C00344', 'C0451

In [11]:
document = libsbml.readSBMLFromFile(os.path.join(dataDirectory, organism, f"mmu00590-sbml.xml"))
model = document.getModel()

In [24]:
for i in range(model.getNumReactions()):
    reaction = model.getReaction(i)
    if reaction.name=="rn:R07053":
        print(reaction.getListOfModifiers()[0])

<ModifierSpeciesReference mod_rnR07053 "mod_rnR07053">
